# Getting started with yieldplotlib

## Introduction
This tutorial will walk you thorugh some of the basic functionality of yieldplotlib to get you started on generating your own yield visualizations. To run yieldplotlib you need to have at least one AYO output folder, EXOSIMS output folder, or yield input package (YIP). Sample data has been provided in the tutorials folder of this repository for demonstrative purposes. 

You can also run series of pre-made plots for yield outputs which can be found in the yieldplotlib/src/scripts folder, or accessed through the yieldplotlib pipeline and command line interface.

### Imports

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt

import yieldplotlib as ypl
from yieldplotlib.load.ayo_directory import AYODirectory
from yieldplotlib.load.exosims_directory import EXOSIMSDirectory

### Loading Yield Data

 Yieldplotlib uses a key map to equivocate parameters between EXOSIMS and AYO (and potential future yield codes). This key map is pulled daily from a collaborative Google Sheet to the GitHub repository in the form of a key_map.csv file. To see all the possible named parameters that can be plotted in yieldplotlib, consult documentation or the first column of that CSV file/Google Sheet. 

In order to convert this CSV to the key_map.py file that is needed by yieldplotlib to parse the yield output directories, the user should run:

`python generate_key_map.py --csv key_map.csv`

This will populate a key_map.py file containing all the proper mappings. Now we can load our sample AYO and EXOSIMS data using the AYODirectory and EXOSIMSDirectory classes.  

In [ ]:
ayo_folder = Path("sample_data/ayo")
exosims_folder = Path("sample_data/exosims")

In [ ]:
ayo = AYODirectory(ayo_folder)
exosims = EXOSIMSDirectory(exosims_folder)

Now that we have loaded our directories, we can display the file structure to see what is contained in our yield outputs at a glance.

In [ ]:
print(exosims.display_tree())
print(ayo.display_tree())

## Accessing Data and Generating Plots 
We have our data and can now start accessing the data and generating plots!

### Get Data
Data can be accessed from the AYODirectory and EXOSIMSDirectory using the .get() function which takes the named yieldplotlib keys as input and outputs the corresponding values. This can be in the form of singular values or arrays.

In [ ]:
# Pupil diameter
ayo_pup = ayo.get("pupil_diam")
exo_pup = exosims.get("pupil_diam")
print(f"Pupil diameters are:\nAYO: {ayo_pup} \nEXOSIMS: {exo_pup}")

# ExoEarth yields
ayo_yield = ayo.get("yield_earth")
exo_yield = exosims.get("yield_earth")
print(f"\nExoEarth yields are:\nAYO: {ayo_yield} \nEXOSIMS: {exo_yield}")

# Target star distances
ayo_dist = ayo.get("star_dist")
exo_dist = exosims.get("star_dist")
print(f"\nStellar distances (pc) are:\nAYO: {ayo_dist} \nEXOSIMS: {exo_dist}")

### Generic Plots

yieldplotlib expands directly on matplotlib to utilize its functionality and to make plot customization intuitive for those who have familiarity with matplotlib and its features. Below are examples of some of the generic yieldplotlib plots that one can make.

In [ ]:
# Histograms

fig, ax = plt.subplots(figsize=(8, 6))
ax.ypl_hist(exosims, x="star_dist", bins=20, alpha=0.7, label="EXOSIMS Stars")
ax.ypl_hist(ayo, x="star_dist", bins=20, alpha=0.7, label="AYO Stars")
ax.set_title("Distribution of Star Distances")

In [ ]:
# Scatter Plots

fig, ax = plt.subplots()
data = ax.ypl_scatter(ayo, x="star_dist", y="star_L", c="star_comp_det")
ax.set_title("HZ Completeness (AYO)")
plt.colorbar(data)

fig, ax = plt.subplots()
data = ax.ypl_scatter(exosims, x="star_dist", y="star_L", c="star_comp_det")
ax.set_title("HZ Completeness (EXOSIMS)")
plt.colorbar(data)

### Comparative Plots 

Instead of plotting datasets individually, for comparison purposes it is often desired to plot different yield outputs in a single figure. Here the suite of available yieldplotlib comparative plots should be used. Lets see two examples plotting the same HZ completeness plot as above. 

In [ ]:
# Two yield runs on same set of axes.

fig, ax = plt.subplots()
ax.set_title("HZ Completeness")
data = ypl.compare(
    ax,
    [exosims, ayo],
    x="star_dist",
    y="star_L",
    plot_type="scatter",
    c="star_comp_det",
)

In [ ]:
# Two yield runs on different sets of axes.

ypl.multi(
    [exosims, ayo],
    x="star_dist",
    y="star_L",
    plot_type="scatter",
    c="star_comp_det",
    figsize=None,
    suptitle=None,
    layout=None,
    sharex=True,
    sharey=True,
    titles=["EXOSIMS", "AYO"],
)

Additionally, we can run more complex analyses on multi-dimensional grids of parameter space. 

In [ ]:
# Plot a grid of parameters against each other.

ypl.xy_grid(
    [ayo],
    ["star_L", "star_dist"],
    ["MV", "star_dist"],
    plot_type="scatter",
    figsize=None,
    suptitle="Stellar Parameters",
    legend=True,
    sharex=False,
    sharey=False,
    titles=None,
)

In [ ]:
# Plot multiple panels with different specifications for each yield run.

spec1 = {
    "x": "star_L",
    "y": "star_dist",
    "plot_type": "scatter",
    "c": "star_comp_det",
    "alpha": 0.7,
}
spec2 = {
    "x": "star_L",
    "y": "star_dist",
    "plot_type": "scatter",
    "c": "exp_time_char",
    "alpha": 0.7,
}
specs = [spec1, spec2]

fig, axes = ypl.panel(
    [exosims, ayo],
    *specs,
    figsize=None,
    suptitle=None,
    layout=None,
    sharex=True,
    sharey=True,
    titles=["HZ Completeness", "Exp Time Char", "", ""],
)